## Homework 3 - Supervised Learning II - MDS Computational Linguistics

### Assignment Topics
- Recurrent Neural Networks
- Long Short-Term Memory
- Saving and Loading NN models using Pytorch
- Very-short answer questions

### Software Requirements
- Python (>=3.6)
- PyTorch (>=1.2.0) 
- Jupyter (latest)

### Submission Info.
- Due Date: 1/30/21 11:59 PM PST

## Getting Started

In [1]:
# all the necessary imports
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch import optim
import torchtext
from torchtext.data import Field, LabelField
from torchtext.data import TabularDataset
from torchtext.data import Iterator, BucketIterator

In [2]:
# set the seed
manual_seed = 572
torch.manual_seed(manual_seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


## Tidy Submission

rubric={mechanics:1}

To get the marks for tidy submission:
- Submit the assignment by filling in this jupyter notebook with your answers embedded
- Be sure to follow the [general lab instructions](https://ubc-mds.github.io/resources_pages/general_lab_instructions)

### T3 - CNN's for text classification

Convolutional Neural Networks (CNNs) are essentially a special case of a normal feed forward network, instead of being "densely" connected (note you may see people refer to Feed Forward networks as "dense layers"), nodes in CNNs connect to a smaller set of nodes, defined by the "filters size" of the network. CNNs thus have a smaller local windows to look at data, but make up for it by generally using many additional filters, which might be able to learn different aspects of the data. These networks turn out to be extremely useful for processing images, audio, and anything with some sort of spatial properties to the data.

It turns out they can also be used for any sentence classification task. Words in a sentence it turns out have a sort of 1D spatial ordering, which means some classification tasks can benefit from this CNNs ability to operate over the length of the sequence. In addition, because of the sparsity of the connections, you end up being able to make much smaller networks that retain a great deal of power.

<img src="images/cnn.png" alt="cnn" title="CNN vs FF-net comparison" />

<p style="text-align: center;">Top: A CNN with filter size of 3, Bottom: A Feedforward neural net. (From Goodfellow et al. 2016)</p>

This illustrates a single size 3 filter, your network might have many additional filters that would all pass over the length of the data, each one producing an output channel to be passed to the next part of the network.  

<img src="images/1dcnn.png" alt="1dcnn" title="Pytorch 1DCNN overview" />

*We'll get into 2D CNNs later (as an additional topic in COLX 585) but here is a quick taste for how you could use a 1D CNN in the context of text-based NLP tasks.*


#### Pytorch 1D CNNs and max-pooling

Here's a quick example of how these networks function:

In [3]:

x = torch.rand((2,5,10))   # batch size 2 with length 10 and 5 dim embedding.

in_dim = 5
filters = 4

cnn1d = nn.Conv1d(in_dim,filters,kernel_size=3,padding=1) 
max_pool = nn.MaxPool1d(kernel_size=3, padding=1)
activation = nn.ReLU()
x = cnn1d(x)
x = activation(x)
print(x)
print("Take the highest value in each window using max pool")
print(max_pool(x))


tensor([[[0.0115, 0.1652, 0.0000, 0.0210, 0.1613, 0.3903, 0.0000, 0.1766,
          0.1741, 0.2820],
         [0.0000, 0.0000, 0.0083, 0.1430, 0.0000, 0.0000, 0.2834, 0.0585,
          0.2041, 0.0000],
         [0.0456, 0.4565, 0.1949, 0.3690, 0.0888, 0.2262, 0.0732, 0.5068,
          0.5604, 0.3049],
         [0.0000, 0.0000, 0.0170, 0.1959, 0.0000, 0.0990, 0.0000, 0.0000,
          0.0000, 0.0000]],

        [[0.2387, 0.2856, 0.0000, 0.0000, 0.2042, 0.3228, 0.0630, 0.0000,
          0.0000, 0.2929],
         [0.0556, 0.2714, 0.0000, 0.0000, 0.0000, 0.1017, 0.1376, 0.1463,
          0.0000, 0.0000],
         [0.1951, 0.6214, 0.0341, 0.6379, 0.3405, 0.5166, 0.4090, 0.2046,
          0.3178, 0.5490],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.2466]]], grad_fn=<ReluBackward0>)
Take the highest value in each window using max pool
tensor([[[0.1652, 0.1613, 0.3903, 0.2820],
         [0.0000, 0.1430, 0.2834, 0.2041],
         [0.4565, 0.3690

Here we've used a 1D CNN and max pooling to "summarize" our data, boiling a length 10 series to only 4 items. One could repeat combinations of CNNs, activation functions, with and without pooling layers to build up a network in the same way that we have worked with FF nets.

But first one quick check about 1D CNNs

#### 1D CNN parameters
rubric={reasoning:1}  
  
Including bias weights how many trainable parameters are in our example CNN (think about how many weights would be needed to operate over your kernel size and each segment of data)?  Explain your reasoning in your answer, and consider checking it by printing out those weights/biases.


In [5]:
count = 0
for param in cnn1d.parameters():
    if param.requires_grad==True:
        count+= param.numel()
print(count)

# 4 filters X 5 embedding size X 3 filter size + 4 filters X 1 bias per filter = 64 

64


*Your Answer Here*

#### Size through CNNs
rubric={accuracy:1, reasoning:1}

CNNs can be a little tricky because as the data passes through them the dimensions might change based on number of filters, padding, and two other things we won't talk about now: stride and dilation. MaxPooling also quickly can decrease the size of the data. This is useful especially as a way to "feature extract" or "dimensionality reduction" but it's important to make sure we get the right output dimensions. 

For an tensor with batch size of 10, embedding dimension of 25, and length 30 that is fed to a 1D CNN + Maxpool combination, what parameters of the 1D CNN (kernelsize, number of filters, padding) and MaxPool (kernel_size, padding) would we need to get a output that has an output embedding dimension of 5 and a length of 10?


*Your Answer Here*

In [6]:
## Test your answer by making a random tensor of the appropriate size, 
## passing it through your proposed CNN+Maxpool and printing out the final size.

## Your Code Here ##

x = torch.randn((10,25,30))  # our example tensor
cnn1 = nn.Conv1d(25,5,kernel_size=3,padding=1)
maxpool = nn.MaxPool1d(kernel_size=3,padding=1)

print(x.size())
print(maxpool(cnn1(x)).size())

torch.Size([10, 25, 30])
torch.Size([10, 5, 10])


Note we can ignore batch (N) in the problem, there should be a number of different ways to get the same output Length (L) depending on how you set the kernel_sizes and padding, but giving the maxpool a kernel size of 3 is the easiest way.

Output hidden/embedding dimension (H) is dependent on the number of filters in the CNN, which means it must be set to 5.

#### 1D CNN Model for Sentiment Analysis
rubric={accuracy:3, quality:1}

Based on our tutorial code that we've used for RNNs, modify our network to use 1D cnns instead of FF networks. We'll search over the architecture in the next question, but for now just make it as a basic 2 layer CNN network with 5 filters kernel size of 3 and using ReLU activation (no max Pooling). 

In [18]:
import torchtext
from torchtext.data import Field, LabelField
from torchtext.data import TabularDataset

# define the white space tokenizer to get tokens
def tokenize_en(tweet):
    """
    Tokenizes English tweet from a string into a list of strings (tokens)
    """
    return tweet.strip().split()

# define the TorchText's fields
TEXT = Field(sequential=True, tokenize=tokenize_en, lower=True)
LABEL = Field(sequential=False, unk_token = None)


train, val, test = TabularDataset.splits(
    path="./data/sentiment-twitter-2016-task4/", # the root directory where the data lies
    train='train.tsv', validation="dev.tsv", test="test.tsv", # file names
    format='tsv',
    skip_header=False, # if your tsv file has a header, make sure to pass this to ensure it doesn't get proceesed as data!
    fields=[('tweet', TEXT), ('label', LABEL)])

TEXT.build_vocab(train, min_freq=3) # builds vocabulary based on all the words that occur at least twice in the training set
LABEL.build_vocab(train)

train_iter, val_iter, test_iter = BucketIterator.splits(
 (train, val, test), # we pass in the datasets we want the iterator to draw data from
 batch_sizes=(4,64,64),
 sort_key=lambda x: len(x.tweet), 
 sort=True,
# A key to use for sorting examples in order to batch together examples with similar lengths and minimize padding. 
 sort_within_batch=True
)

VOCAB_SIZE = len(TEXT.vocab.stoi)

WORD_VEC_SIZE=300
# Note, the parameters to Embedding class below are:
# num_embeddings (int): size of the dictionary of embeddings
# embedding_dim (int): the size of each embedding vector
# For more details on Embedding class, see: https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/sparse.py


class ConvNet(nn.Module):
  
  def __init__(self, VOCAB_SIZE, WORD_VEC_SIZE,output_size, config):  
    """Build a 1D CNN for classification.

    Keyword arguments:
    config -- a dictionary of parameters for the model
        layers -- a tuple containing the filter#, filtersize, and dropout% per layer
        nonlin -- nonlinearity
        pool_mode -- None, nn.MaxPool1d, or nn.AvgPool1d
    """
    super(ConvNet, self).__init__()
    self.embedding = nn.Embedding(VOCAB_SIZE, WORD_VEC_SIZE, sparse=True)
    self.embedding.weight.data.normal_(0.0,0.05)
    self.layers = nn.ModuleList()
    self.nonlin = config["nonlin"]
    for i in range(len(config["layers"])):
        if i == 0:
            self.layers.append(nn.Conv1d(WORD_VEC_SIZE, config["layers"][i][0], config["layers"][i][1], padding=(config["layers"][i][1]-1)//2))
        else:
            self.layers.append(nn.Conv1d(config["layers"][i-1][0],config["layers"][i][0],config["layers"][i][1], padding=(config["layers"][i][1]-1)//2))      
        self.layers.append(self.nonlin)
        self.layers.append(nn.Dropout(p=config["layers"][i][2]))
        if config["pool_mode"]:
                self.layers.append(config["pool_mode"](kernel_size=2,padding=1))
    self.max_layer = nn.AdaptiveMaxPool1d(1)
    self.output = nn.Linear(config["layers"][-1][0], output_size)
    self.softmax = nn.LogSoftmax(dim=1) 

  def forward(self, x):
    x = self.embedding(x.permute(1,0)).permute(0,2,1)
    for layer in self.layers:
        x = layer(x)
    x = self.max_layer(x).squeeze(dim=-1)
    x =self.softmax(self.output(x))  # [N,H]
    
    return x



from sklearn.metrics import accuracy_score
def train(loader,model,criterion,optimizer,device):
    total_loss = 0.0
    # iterate throught the data loader
    num_sample = 0
    for batch in loader:
        # load the current batch
        batch_input = batch.tweet
        batch_output = batch.label
        
        batch_input = batch_input.to(device)
        batch_output = batch_output.to(device)
        # forward propagation
        # pass the data through the model
        model_outputs = model(batch_input)
        # compute the loss
        cur_loss = criterion(model_outputs, batch_output)
        total_loss += cur_loss.item()

        # backward propagation (compute the gradients and update the model)
        # clear the buffer
        optimizer.zero_grad()
        # compute the gradients
        cur_loss.backward()
        # update the weights
        optimizer.step()

        num_sample += batch_output.shape[0]
    return total_loss/num_sample

# evaluation logic based on classification accuracy
def evaluate(loader,model,criterion,device):
    all_pred=[]
    all_label = []
    with torch.no_grad(): # impacts the autograd engine and deactivate it. reduces memory usage and speeds up computation
        for batch in loader:
             # load the current batch
            batch_input = batch.tweet
            batch_output = batch.label

            batch_input = batch_input.to(device)
            # forward propagation
            # pass the data through the model
            model_outputs = model(batch_input)
            # identify the predicted class for each example in the batch
            probabilities, predicted = torch.max(model_outputs.cpu().data, 1)
            # put all the true labels and predictions to two lists
            all_pred.extend(predicted)
            all_label.extend(batch_output)
            
    accuracy = accuracy_score(all_label, all_pred)
    return accuracy



#### 1D CNN Performance
rubric={accuracy:2, quality:1}

Based on our initial network we'd like to compare how depth matters vs number of filters in a given layer. For this section (consider spliting up the trials similar to Lab 1, with group members running a set on different random start states), perform a random search (total trials: 20) over the parameters space (number of layers, kernel size, number of filters, activation, etc.) and report your results (ranges searched, best performance) on the sentiment task. Hint: Feel free to use Skorch + sklearn + scipy.stats (for distributions)

In [38]:
# your code here.

import scipy.stats
import numpy as np


MAX_EPOCHS=20 # can be higher

np.random.seed(manual_seed)  #Note sets for both Numpy and Scipy 
torch.manual_seed(manual_seed) 

def random_search(num_iter):    
    results = []
    for i in range(num_iter):
        
        #generate a number of layers
        layers = []
        layer_num = scipy.stats.randint.rvs(1,5)
        for _ in range(layer_num):
            # appending (FILTERS# , FILTERSIZE , DROPOUT%) per each layer
            layers.append((scipy.stats.randint.rvs(10,200),np.random.choice([1,3,5,7]),scipy.stats.uniform.rvs()))
        config = {
            #define hyperparameters here
            "layers": layers,
            "lr": scipy.stats.loguniform.rvs(10**-3,1),
            #"l2": scipy.stats.loguniform.rvs(10**-5,10**-2),
            "nonlin" : np.random.choice([nn.ReLU(),nn.Tanh()]),
            "pool_mode" : np.random.choice([None,nn.MaxPool1d,nn.AvgPool1d])
        }
        
        print("new config")
        print(config)
        model = ConvNet(output_size=3, config=config, VOCAB_SIZE=VOCAB_SIZE, WORD_VEC_SIZE=WORD_VEC_SIZE)
        model.to(device)
        criterion = nn.NLLLoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=config["lr"])

    
        max_val = 0
        best_epoch = 0
        for epoch in range(MAX_EPOCHS):
        # train the model for one pass over the data
            train_loss = train(train_iter,model,criterion,optimizer,device)  
        # compute the training accuracy
            train_acc = evaluate(train_iter,model,criterion,device)
        # compute the validation accuracy
            val_acc = evaluate(val_iter,model,criterion,device)
            if val_acc > max_val:
                max_val = val_acc
                best_epoch = epoch+1
        # print the loss for every epoch
            print('Epoch [{}/{}], Loss: {:.4f}, Training Accuracy: {:.4f}, Validation Accuracy: {:.4f}'.format(epoch+1, MAX_EPOCHS, train_loss, train_acc, val_acc))
        results.append((max_val,best_epoch,config))
        print("Best validation score for iterations #{}: {}".format(i,max_val))
    return results


In [ ]:
random_search(20)

Highest config was around ~50.6% on my run.

#### 1D CNN summary
rubric={reasoning:1}

Based on the performance on the task how does the CNN compare to the Feed Forward network from Lab 2? What factors seemed to be most important to the performance of the model (number of filters, depth, size of filters...? ) 

* You answer here *

Because the results from your random search might be a bit different, you might have found different configurations that worked better. Ultimately as long as you are able to compare the two, and identify important factors in getting the CNN config to work well you will receive full credit.

## Exercise 1: Building a Recurrent Neural Network

In this exercise, we provide a corpus from the [CL-Aff shared task](https://sites.google.com/view/affcon2019/cl-aff-shared-task?authuser=0). HappyDB is a dataset of about 100,000 `happy moments` crowd-sourced via Amazon’s Mechanical Turk where each worker was asked to describe in a complete sentence `what made them happy in the past 24 hours`. Each user was asked to describe three such moments. 
In this exercise, we focus on sociality classification. We only use labelled dataset which include 10,562 labelled samples. 

We have already preprocessed (tokenization, removing URLs, mentions, hashtags and so on) the tweets and placed it under ``data/happy_db`` folder in three files as ``train.tsv``, ``dev.tsv`` and ``test.tsv``.

#### 1.1 Write code to  define a `whitespace_tokenizer` whose input is a tweet text.

rubric={accuracy:1}

In [ ]:
def whitespace_tokenize(text):
    # your code goes here 
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return text.strip().split()

#### 1.2 Wrote code to define `TorchText's Fields` handle how data should be processed.  
Hint: You need 2 Fields `TEXT` and `LABEL` for tweet text and label respectively. 

* The tokenizer is the whitespace_tokenizer in 1.1.
* Use the truecase of words. 

rubric={accuracy:2} 

In [ ]:
# your code goes here 
TEXT = Field(sequential=True, tokenize=whitespace_tokenize, lower=False)
LABEL = Field(sequential=False, unk_token = None)

#### 1.3 Write code to use `TabularDataset class`  and `Fields` to process the tsv files (train, dev, and test). 

Hint 1: `Fields` will call tokenizer, and convert tokens to numerical index. 

Hint 2: Use `TabularDataset.splits(...)` to load train, dev, and test sets.

rubric={accuracy:2}

In [ ]:
# your code goes here 
train, val, test = TabularDataset.splits(
               path="./data/happy_db/", # the root directory where the data lies
               train='train.tsv', validation="dev.tsv", test="test.tsv", # file names
               format='tsv',
               skip_header=True, # if your tsv file has a header, make sure to pass this to ensure it doesn't get proceesed as data!
               fields=[('tweet', TEXT), ('label', LABEL)])

#### 1.4  Write code to build your vocabulary to map words and labels to integers. The maximum size of vocabulary is 5,000 (not count `<pad>` and `<unk>` in).
Hint: Use `build_vocab` to build vocabularies for `TEXT` field and `LABEL` field respectively. 

rubric={accuracy:2}

In [ ]:
# your code goes here 
TEXT.build_vocab(train, max_size=5000)
LABEL.build_vocab(train)

#### 1.5 Write code to print the sizes of your two vocabularies (i.e., `TEXT` and `LABEL`) individually. 

rubric={accuracy:2}

In [ ]:
# your code goes here 
print("Vocabulary size of TEXT:",len(TEXT.vocab.stoi))
print("Vocabulary size of LABEL:",len(LABEL.vocab.stoi))

#### 1.6 Write code to construct the Iterators to get the train, dev, and test splits. Use `BucketIterator` to initialize the Iterators for the train, dev, and test data.
* apply same batch size of 32 on train, dev, and test set.
* Samples are sorted by length.
* Sort samples within each batch

Hint: Use `BucketIterator.splits(...)`

rubric={accuracy:2}

In [ ]:
# your code goes here 
train_iter, val_iter, test_iter = BucketIterator.splits(
 (train, val, test), # we pass in the datasets we want the iterator to draw data from
 batch_sizes=(32,32,32),
 sort_key=lambda x: len(x.tweet), 
 sort=True,
# A key to use for sorting examples in order to batch together examples with similar lengths and minimize padding. 
 sort_within_batch=True
)

#### 1.7.1 Write code to create a class called `LSTMmodel` to define a classifier of the task. In this model, you should have:

1. An embedding layer with input dimention equal to the size of your `TEXT` vocabulary, and that represent each token in a 300-dimentional vector.  The parameters of this embedding layer should be randomly initialized with numbers sampled from a normal distribution (mean 0 and variance 0.05).

2. Two uni-directional `LSTM` layers. Each layer has 500 hidden unites.

3. Pass the last hidden of last layer into a `Tanh` activation function, then feed the output of `Tanh` to a linear layer whose dimensionality is equal to the number of classes in the dataset (i.e., 2 in our case).

4. Then, a `LogSoftmax` layer on top of the outcome of `linear layer`.

5. Return the output of `LogSoftmax` layer.

Hint: `Tanh` might not be the ideal function to use, but we want you to explore it. (Usually `ReLU` works well).

rubric={accuracy:8, quality:4}

In [ ]:
# your code goes here 

class LSTMmodel(nn.Module):
  
  def __init__(self, embedding_size, vocab_size, output_size, hidden_size, num_layers):
    # In the constructor we define the layers for our model (same as our previous RNN)
    super(LSTMmodel, self).__init__()
    # word embedding lookup table
    self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_size)
    self.embedding.weight.data.normal_(0.0,0.05) # mean=0.0, mu=0.05
    # core LSTM module
    self.lstm_rnn = nn.LSTM(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers) # input_size, hidden_size, num_layers
    self.activation_fn = nn.Tanh()
    self.linear_layer = nn.Linear(hidden_size, output_size) 
    self.softmax_layer = nn.LogSoftmax(dim=-1)  
  
  def forward(self, x):
    # In the forward function we define the forward propagation logic
    out = self.embedding(x)
    out, (h_state, c_state) = self.lstm_rnn(out) # h_0 initialized to zeros by default
    # classify based on the hidden representation at the last token
    out = out[-1] # unsqueeze converts 1D input (D dimension) into 2D input (1xD) 
    out = self.linear_layer(out)
    out = self.softmax_layer(out) # accepts 2D or more dimensional inputs
    return out

#### 1.7.2 Write code to Instantiate the model class with aforementioned hyper-parameters and print out the model architecture.
rubric={accuracy:3}

Hint: Your output should generally look like the following (with correct names of the model, functions, etc.):
```
modelName (
  (embedding): Embedding(xxx, xxx)
  (lstm_rnn): MODEL(xxx, xxx, num_layers=xxx)
  (activation_fn): Tanh()
  (linear_layer): Linear(in_features=xxx, out_features=xxx, bias=True)
  (softmax_layer): somethingRelatedToSoftmax()
)
```

In [ ]:
# your code goes here 
EMBEDDING_SIZE = 300 
VOCAB_SIZE = 5002
NUM_CLASSES = 2
HIDDEN_SIZE = 500
NUM_LAYERS = 2
model = LSTMmodel(EMBEDDING_SIZE, VOCAB_SIZE, NUM_CLASSES, HIDDEN_SIZE, NUM_LAYERS)
model = model.to(device)
print(model)

**Create an optimizer for training**

In [ ]:
LEARNING_RATE = 0.1
criterion = nn.NLLLoss()
# create an instance of SGD with required hyperparameters
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

#### 1.8.1 How many learnable (or updatable) parameters are present in the model defined in 1.7. Compute the result by writing code.
rubric={accuracy:1}

In [ ]:
# your code goes here 
count = 0
for p in model.parameters():
    count += p.numel()
print("the number of parameters:",count)

#### 1.8.2 OPTIONAL QUESTION: How many megabyte memory will this model use? Please show your work. 

Hint1: Each parameter is a `torch.float32` tensor which is 32-bit floating point. 

Hint2: All the parameters of this model are learnable (or updatable) parameters.

rubric={spark:2}

#### Write your answer here.
$5109602 * 32 bits = 163507264 bits$

$1bit = 1.25e-7 Megabyte$

$163507264 bits = 20.438408 Megabyte$ 

Actully, you can find the size of the checkpoint (i.e., model_23.pt) is 20.4 MB.

**To facilitate your work for the next questions (1.9 and 1.10), we provide two functions for training and evaluation.**

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

def train(loader):
    total_loss = 0.0
    # iterate throught the data loader
    num_sample = 0
    for batch in loader:
        # load the current batch
        batch_input = batch.tweet
        batch_output = batch.label
        
        batch_input = batch_input.to(device)
        batch_output = batch_output.to(device)
        # forward propagation
        # pass the data through the model
        model_outputs = model(batch_input)
        # compute the loss
        cur_loss = criterion(model_outputs, batch_output)
        total_loss += cur_loss.item()

        # backward propagation (compute the gradients and update the model)
        # clear the buffer
        optimizer.zero_grad()
        # compute the gradients
        cur_loss.backward()
        # update the weights
        optimizer.step()

        num_sample += batch_output.shape[0]
    return total_loss/num_sample

# evaluation logic based on classification accuracy
def evaluate(loader):
    all_pred=[]
    all_label = []
    with torch.no_grad(): # impacts the autograd engine and deactivate it. reduces memory usage and speeds up computation
        for batch in loader:
             # load the current batch
            batch_input = batch.tweet
            batch_output = batch.label

            batch_input = batch_input.to(device)
            # forward propagation
            # pass the data through the model
            model_outputs = model(batch_input)
            # identify the predicted class for each example in the batch
            probabilities, predicted = torch.max(model_outputs.cpu().data, 1)
            # put all the true labels and predictions to two lists
            all_pred.extend(predicted)
            all_label.extend(batch_output)
            
    accuracy = accuracy_score(all_label, all_pred)
    f1score = f1_score(all_label, all_pred, average='macro') 
    return accuracy,f1score

#### 1.9 Please read [Pytorch documentation](https://pytorch.org/tutorials/beginner/saving_loading_models.html) and write code to load the trained model checkpoint from directory: `./ckpt/model_23.pt`.

Note this ckpt is a dictionary which includes three keys: epoch, model_state_dict, and optimizer_state_dict.
The model parameters are the values of "model_state_dictm".



rubric={accuracy:2}

In [ ]:
# your code goes here 
checkpoint = torch.load("./ckpt/model_23.pt")
model.load_state_dict(checkpoint['model_state_dict'])

#### 1.10 Write code to evaluate the trained model on test set and report the test accuracy and F-score. 
Report the performance of the trained model on the test set is XX.XXX% on accuracy (fill in your accuracy).
#### Because there were some issues with the pre-trained model, as long as you loaded it, you'll receive credit. Should be ~50%
Report the performance of the trained model on the test set is XX.XXX on F1_score (fill in your F1_score).

rubric={accuracy:2,quality:1}

In [ ]:
# your code goes here 
print(evaluate(test_iter))

# 2 Short Answer Questions

### 2.1 Non-Linearity Review
rubric={reasoning:2}

There are a number of different non-linearities that can be used in our neural networks, for instance: **Sigmoid, tanh, and ReLU**. Different variations and tweaks to these non-linearities get introduced fairly frequently, and it pays to have a sense of why you might pick one non-linearity for your network instead of another. **Explain how these three (Sigmoid, tanh, and ReLU) are different and what might be some pros and cons of using them in a neural network**. (There are some great blog/quora posts talking about this topic, but **if you are going to be summarizing from a post or other online resource please include a link to the resource**).


Hint: Your answer should be a maximum of 2-3 paragraphs. Short answers are just fine.

#### Write your answer here.

- Sigmoid and Tanh can both run into issues where the gradients get very small when input gets close to (+inf or -inf), this is the vanishing gradient problem.

- Tanh is between (-1,1) which makes training a little bit nice than sigmoid.

- ReLU is super fast to compute and doesn't have the vanishing gradient issue.

There are of course other things you could have found in your research, mainly looking that you are able to look through some resources and weigh the pros and cons of the different non-linearities.

### 2.2 ReLU Variations
rubric={reasoning:1}

PyTorch supports several other non-linearities, find two variations on ReLU that PyTorch implements (see https://pytorch.org/docs/stable/nn.html) and explain how they are different from standard ReLU. 

#### Write your answer here.
Picking one as an example: 

- Leaky ReLU is like ReLU but allows for some small (tiny!) value when the input is negative. This might be important if you still want to differentiate between negative input values and could potentially be slightly more powerful than ReLU at the expense of being slightly more expensive to compute. 

There are a lot of other choices, but just consider what might be some benefits for these other "advanced" activation functions?

### 2.3 Wait why are we using Logs?
rubric={reasoning:1}

What is the purpose of taking logs of probabilities, as in the case of NLLLoss and LogSoftmax?

Hint: A short answer is just fine.


#### Write your answer here.

You can run into underflow issues with probabilities. likewise logs of probabilities allow you to calculate things in terms of sums (cheap) as opposed to multiplication, potentially allowing for faster computation of values. If you just care about the biggest probability, the biggest log probability is still going to be larger than the rest, so we can do everything with logs that we could with non-logs.

### 2.4 Softmax
rubric={reasoning:2}

For a multiclass problem with a Softmax layer why might we care about values corresponding to more than one of these classes?

Hint: Sometimes knowing what the top n most likely classes is of interest. Why?

#### Write your answer here.

There are a couple of reasons why you might care about the other numbers though, one reason might be if you want to say handle ambiguous situations slightly differently, say you have the top two classes close together, but both fairly low, in some situations you might want to handle things without just predicting the top class.

In machine translation you will need to decode the probability outputs to determine what the most likely translation over the course of the sequence. Because each timestep is not independent of earlier timesteps you'll sometimes need to look at outputs other than the most likely output for each timestep. One example of how this is done is using something called beam search, where you keep track of the top-N output sequencences during the decoding.  

### 2.5 RNNs
rubric={reasoning:1}

RNNs are quite powerful for dealing with certain types of data (such as sequences), however, they have some drawbacks. What are some of these drawbacks? (List two drawbacks)

Hint: Look at RNN slides, including the last few slides.


#### Write your answer here.

Clear issues include dealing with long sequences and the runtime of the algorithm (it can't be parallelized).

### 2.6 LSTMs vs RNNs

rubric={reasoning:2}

LSTMs help alleviate one of the issues with RNNs (see 2.5). What do they alleviate? Do you see any problems that the model might still have (just reason generally about the model based on your understanding).

Hint: Short answer is good.


#### Write your answer here.

LSTMs are better at sequence length issues, [although attention based methods which we will get to later in the semester turn out to be much better. They will still run into issues with very very long sequence lengths, although now they are able to learn to pay attention to certain things over the course of the sequence. They still have the same weakness in terms of speed, as they can't be parallelized.